# Task 3.4 – Model Modification

Import the SVM classification model from Task 3.0.3.

1. Add a new endpoint `GET /retrain` which expects the two model parameters `C` and `gamma`, trains the model with this parameters, and returns a JSON-formatted object with a single attribute `accuracy`.
* Add a new endpoint `GET /retrain_grid` which expects the for both model parameters `C` and `gamma` each `*_start`, `*_end` and `*_step`, trains the model with the resulting parameter ranges (using a GridSerarchCV), and returns a JSON-formatted object the attributes `accuracy` as well as `best_params` containg the parameters for `C` and `gamma` (see Task 3.3). 
*Check with:
http://localhost:5000/retrain_grid?C_start=1000&C_end=2000&C_step=1000&gamma_start=0.01&gamma_end=0.07&gamma_step=0.02*

In [ ]:
from flask import Flask, jsonify, request
import pandas as pd

In [ ]:
df = pd.read_csv('./data/lfw_people.csv', index_col = 0)

df['target'] = df['target'].astype('category')
target_names = df['target'].cat.categories
df['target'] = df['target'].cat.codes

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

from sklearn.decomposition import PCA

pca = PCA(n_components=150, svd_solver='randomized', whiten=True).fit(X_train)

X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
app = Flask(__name__)

In [ ]:
# 3.4-1
@app.route('/retrain')
def retrain():
    C = float(request.args.get('C'))
    gamma = float(request.args.get('gamma'))
    
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV

    clf = SVC(kernel='rbf', class_weight='balanced', C=C, gamma=gamma)
    clf = clf.fit(X_train_pca, y_train)

    y_pred = clf.predict(X_test_pca)

    return jsonify({'accuracy': clf.score(X_test_pca, y_test)})

In [ ]:
# 3.4-2
@app.route('/retrain_grid')
def retrain_grid():
    C_start = float(request.args.get('C_start'))
    C_end = float(request.args.get('C_end'))
    C_step = float(request.args.get('C_step'))

    gamma_start = float(request.args.get('gamma_start'))
    gamma_end = float(request.args.get('gamma_end'))
    gamma_step = float(request.args.get('gamma_step'))
    
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV
    
    import numpy as np
    
    param_grid = {'C': np.arange(C_start, C_end + C_step, C_step),
                  'gamma': np.arange(gamma_start, gamma_end + gamma_step, gamma_step)}
    
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)

    y_pred = clf.predict(X_test_pca)

    return jsonify({'accuracy': clf.score(X_test_pca, y_test),
                   'best_params': clf.best_params_})

In [ ]:
app.run(debug=True, use_reloader=False, host='0.0.0.0')

*Note: To restart the app please interrupt the kernel and execute **all** cells again.*